# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt','wordnet'])
import re
import pickle
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /Users/suman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/suman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('response',engine)
X = df['message']
Y = df.iloc[:,5:]
Y.fillna(0,inplace=True)
np.any(np.isnan(Y))

False

In [3]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    #tokens = word_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(t)
    return tokens

In [5]:
t='Haitians in U.S. watch, earthquake #news and worry http bit.ly 6PkPGv News #CNN favorited false in_reply_to_user_id null in_reply_to_status_id null in_reply_to_screen_name null source <a href= http ping.fm rel= nofollow >Ping.fm< a>'
#soup=BeautifulSoup(t,'html5lib')
#print(soup.get_text())
tokenize(t)

#tokenizer = RegexpTokenizer(r'\w+')
#tokenizer.tokenize(t)


['Haitians',
 'in',
 'U',
 'S',
 'watch',
 'earthquake',
 'news',
 'and',
 'worry',
 'http',
 'bit',
 'ly',
 '6PkPGv',
 'News',
 'CNN',
 'favorited',
 'false',
 'in_reply_to_user_id',
 'null',
 'in_reply_to_status_id',
 'null',
 'in_reply_to_screen_name',
 'null',
 'source',
 'a',
 'href',
 'http',
 'ping',
 'fm',
 'rel',
 'nofollow',
 'Ping',
 'fm',
 'a']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
model = model_pipeline()
model.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = model.predict(X_test)
category_names=list(Y.columns)    
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

/Users/suman/opt/miniconda3/envs/disaster_reponse_pipeline_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Category: related 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1258
           1       0.75      1.00      0.86      3940
           2       0.00      0.00      0.00        46

    accuracy                           0.75      5244
   macro avg       0.25      0.33      0.29      5244
weighted avg       0.56      0.75      0.64      5244

Accuracy of                   related: 0.75
Category: request 
               precision    recall  f1-score   support

           0       0.83      1.00      0.91      4337
           1       0.00      0.00      0.00       907

    accuracy                           0.83      5244
   macro avg       0.41      0.50      0.45      5244
weighted avg       0.68      0.83      0.75      5244

Accuracy of                   request: 0.83
Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5223
           1       0.00      0.00    

Category: shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        25

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

Accuracy of                     shops: 1.00
Category: aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      5186
           1       0.00      0.00      0.00        58

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.99      0.98      5244

Accuracy of               aid_centers: 0.99
Category: other_infrastructure 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      5009
           1       0.00      0.00      0.00       235

    accuracy       

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [9]:
!rm ../models/classifier.pkl
pickle.dump(model, open('../models/classifier.pkl', "wb"))

rm: ../models/classifier.pkl: No such file or directory


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.